In [1]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
from langchain_community.document_loaders.csv_loader import CSVLoader
import os  # 환경 변수 설정을 위한 모듈

# OpenAI API 키 설정 (실제 사용 시에는 안전한 방법으로 API 키를 관리해야 함)
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드
api_key = os.getenv("OPENAI_API_KEY")


In [3]:
# 1. CSV 파일 로드
# - CSVLoader를 사용하여 지정된 경로의 CSV 파일을 로드
# - 각 행(row)이 하나의 문서(Document)로 변환됨
loader = CSVLoader(file_path="../data/titanic.csv")
docs = loader.load()
print(f"Row 수: {len(docs)}")

Row 수: 891


In [4]:
# 2. 불러온 문서 내용 확인
# - 특정 문서의 내용을 출력하여 데이터 확인
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


[페이지내용]
PassengerId: 11
Survived: 1
Pclass: 3
Name: Sandstrom, Miss. Marguerite Rut
Sex: female
Age: 4
SibSp: 1
Parch: 1
Ticket: PP 9549
Fare: 16.7
Cabin: G6
Embarked: S

[metadata]
{'source': '../data/titanic.csv', 'row': 10}



In [5]:
# 3. 문서를 작은 청크로 분할
# - RecursiveCharacterTextSplitter를 사용하여 텍스트를 일정 크기의 청크로 나눔
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
chunk_size = 1000  # 적절한 값 조절 필요
chunk_overlap = 100  # 적절한 값 조절 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 문서를 여러 개의 작은 청크로 나눔
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인

청크 개수: 891


In [6]:
# 4. 벡터스토어(Vector Store) 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_24492\1849054279.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


In [7]:
# 5. 문서에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

In [8]:
# 6. 프롬프트 템플릿 정의 (출처 포함)
# - 검색된 문서를 기반으로 질문에 답변하도록 설계
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [9]:
# 7. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0)

In [10]:
# 8. RAG(Retrieval-Augmented Generation) 체인 생성
# - 검색된 문서를 바탕으로 LLM이 답변을 생성하도록 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 검색을 수행하는 객체 입력
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "데이터에 대해서 알려주세요."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음 데이터는 Titanic 데이터셋(titanic.csv)의 일부로 보이며, 각 행은 한 명의 승객 정보를 담고 있습니다.

주요 열(특성)
- PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked

원천 정보
- source: ../data/titanic.csv
- 각 문서의 metadata에 적힌 row: 예) 687, 623, 206, 202 등

사례 항목 요약
- Dakic, Mr. Branko
  - PassengerId: 688
  - Survived: 0
  - Pclass: 3
  - Sex: male
  - Age: 19
  - SibSp: 0, Parch: 0
  - Ticket: 349228
  - Fare: 10.1708
  - Cabin: (없음)
  - Embarked: S

- Hansen, Mr. Henry Damsgaard
  - PassengerId: 624
  - Survived: 0
  - Pclass: 3
  - Sex: male
  - Age: 21
  - SibSp: 0, Parch: 0
  - Ticket: 350029
  - Fare: 7.8542
  - Cabin: (없음)
  - Embarked: S

- Backstrom, Mr. Karl Alfred
  - PassengerId: 207
  - Survived: 0
  - Pclass: 3
  - Sex: male
  - Age: 32
  - SibSp: 1, Parch: 0
  - Ticket: 3101278
  - Fare: 15.85
  - Cabin: (없음)
  - Embarked: S

- Johanson, Mr. Jakob Alfred
  - PassengerId: 203
  - Survived: 0
  - Pclass: 3
  - Sex: male
  - Age: 34
  - SibSp: 0, Parch: 0
  - Ticket: 31

In [12]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 데이터를 가지고 어떤 걸 분석할 수 있을지 아이디어 5개 리스트 형식으로 정리해줘."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음 데이터를 바탕으로 할 수 있는 분석 아이디어 5가지를 제안합니다.

- Survived 예측 모델 구축 아이디어
  - 목적: Survived를 예측하는 분류 모델 만들기(예: 로지스틱 회귀, RandomForest, GradientBoosting).
  - 사용 변수: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, Ticket, Cabin의 여부 등. Age의 결측치는 평균/중간값 대체(imputation)나 모델 내 결측치 처리로 다루기.
  - 추가 엔지니어링: Name에서 Title(예: Mr., Mrs., Miss, Master 등) 추출 후 포함, FamilySize = SibSp + Parch + 1 같은 파생 변수 추가.

- Descriptive 생존 분석 아이디어
  - 목적: 그룹별 생존 비율을 파악.
  - 그룹 기준 예시: Pclass별, Sex별, Embarked별, 가족 구성 규모(FamilySize)별 등.
  - 방법: groupby와 pivot table로 Survived 비율 계산 및 시각화(바 차트 등).

- FamilySize 영향 분석 아이디어
  - 목적: 가족 규모가 생존에 미치는 영향을 파악.
  - 변수: FamilySize(=SibSp + Parch + 1)와 Survived 간 관계 분석.
  - 방법: FamilySize 구간별 생존율 비교, 로지스틱 회귀에서 FamilySize의 계수 해석, 상호작용 효과 탐색(예: FamilySize vs Pclass).

- Fare와 Pclass의 상호작용 분석 아이디어
  - 목적: Fare 수준과 Pclass가 생존에 어떤 영향을 주는지 파악.
  - 방법: Pclass별 Fare 분포 비교, Fare를 분위수(quartile)로 나눠 생존율 비교, Fare와 Survived 간의 상관 또는 로지스틱 회귀에서 Fare의 효과 확인.

- Name에서 Title 추출 및 사회적 지위 분석 아이디어
  - 목적: Name에